<a href="https://colab.research.google.com/github/Jasper-Hewitt/Wind_project/blob/main/mom_327_queries1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## get and split data

In [ ]:
#todo
#fix the max number of tokens, it's actually very little
#補充 answer with the rest of the rows
#make a better while loop that creates different dataframes based on word count, code now a bit ugly
!pip install fitz
!pip install static
!pip install frontend
!pip install pymupdf --upgrade
!pip install openai

In [ ]:
#manually download the data from github and upload it in this notebook
import fitz

doc = fitz.open("mom_327.pdf")

for page in doc:
    text = page.get_text("text")
    with open("output3.txt", "a") as f:
        f.write(text)

In [ ]:
with open('output3.txt', 'r') as file:
    # Read the contents of the file
    contents = file.read()
    print(contents)

In [ ]:
import re
pattern = "。" #。|:
split_contents = re.split(pattern, contents)
# print(sentences)

In [ ]:
split_contents

In [ ]:
len(split_contents)

721

## sentiment search

In [ ]:
#sentiment search
!pip install sentence_transformers

from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
corpus = split_contents
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# add queries, between 3 and 5
queries = ['噪音', '鯨豚', '保育', '專家', '第327次會議']

# Find the closest X sentences of the corpus for each query sentence based on cosine similarity
top_k = min(68, len(corpus))

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop X most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print("Sentence: ", corpus[idx], " Original Sequence: ", idx+1, " (Score: {:.4f})".format(score))

In [ ]:
len(queries)

5

In [ ]:
sentence_scores = {sentence: {query: 0 for query in queries} for sentence in corpus}

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=len(corpus))
    for score, idx in zip(top_results[0], top_results[1]):
        sentence = corpus[idx]
        sentence_scores[sentence][query] = score

In [ ]:
len(sentence_scores)

241

In [ ]:
#adjust to also allow for queries outside the range of 3-5
import pandas as pd

if len(queries)==3:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]))
elif len(queries)==4:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[3]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]+scores[queries[3]]))
elif len(queries)==5:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[3]], scores[queries[4]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]+scores[queries[3]]+scores[queries[4]])) 


if len(queries) == 3:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)
elif len(queries) == 4:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'similarity_score4', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)
elif len(queries) == 5:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'similarity_score4', 'similarity_score5', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)

In [ ]:
df

,text,original_sequence,similarity_score1,similarity_score2,similarity_score3,similarity_score4,similarity_score5,accumulated_score
54,」,55,"tensor(0.5644, device='cuda:0')","tensor(0.5156, device='cuda:0')","tensor(0.6843, device='cuda:0')","tensor(0.6262, device='cuda:0')","tensor(0.4643, device='cuda:0')","tensor(2.8548, device='cuda:0')"
0,1 \n行政院環境保護署環境影響評估審查委員會 \n第 327 次會議紀錄 \n壹、 時間：...,1,"tensor(0.2263, device='cuda:0')","tensor(0.1678, device='cuda:0')","tensor(0.2463, device='cuda:0')","tensor(0.3214, device='cuda:0')","tensor(0.6077, device='cuda:0')","tensor(1.5695, device='cuda:0')"
116,\n10. 研析因應生態衝擊觀測及共同降載機制,117,"tensor(0.2843, device='cuda:0')","tensor(0.3246, device='cuda:0')","tensor(0.3263, device='cuda:0')","tensor(0.3611, device='cuda:0')","tensor(0.2091, device='cuda:0')","tensor(1.5054, device='cuda:0')"
60,」劉委員小如意見略以：「第 1，簡報 P.8\n鯨豚影響減輕對策中，請確認不得啟動新設風機打...,61,"tensor(0.2211, device='cuda:0')","tensor(0.4175, device='cuda:0')","tensor(0.3045, device='cuda:0')","tensor(0.2959, device='cuda:0')","tensor(0.2660, device='cuda:0')","tensor(1.5050, device='cuda:0')"
205,\n4. 最大噪音量容忍值，參考國際海洋噪音管理與對鯨豚類\n影響減輕規範及國內現有研究調...,206,"tensor(0.5108, device='cuda:0')","tensor(0.2924, device='cuda:0')","tensor(0.2042, device='cuda:0')","tensor(0.2462, device='cuda:0')","tensor(0.2351, device='cuda:0')","tensor(1.4887, device='cuda:0')"
...,...,...,...,...,...,...,...,...
173,\n二、 環境影響摘要 \n（一） 海岸地形變遷模擬結果顯示，4 風場離岸風機設置後對\n...,174,"tensor(0.0718, device='cuda:0')","tensor(0.0012, device='cuda:0')","tensor(0.0227, device='cuda:0')","tensor(0.0399, device='cuda:0')","tensor(-0.0029, device='cuda:0')","tensor(0.1328, device='cuda:0')"
235,\n八、 空氣污染 \n施工階段引擎應優先採用低硫氧化物及粒狀污染物等空氣污\n染排放之高...,236,"tensor(0.1150, device='cuda:0')","tensor(-0.0607, device='cuda:0')","tensor(-0.0214, device='cuda:0')","tensor(0.0321, device='cuda:0')","tensor(-0.0239, device='cuda:0')","tensor(0.0412, device='cuda:0')"
91,\n（8） 開發影響範圍侷限於場址附近，對其他國家之環境\n無造成顯著不利影響,92,"tensor(-0.0054, device='cuda:0')","tensor(-0.0386, device='cuda:0')","tensor(0.1006, device='cuda:0')","tensor(-0.0231, device='cuda:0')","tensor(-0.0180, device='cuda:0')","tensor(0.0155, device='cuda:0')"
47,\n9. 開發計畫屬潔淨再生能源風力發電，營運階段於機組\n運轉期間僅以天然風力提供機組運...,48,"tensor(0.0048, device='cuda:0')","tensor(-0.0952, device='cuda:0')","tensor(0.0735, device='cuda:0')","tensor(0.0062, device='cuda:0')","tensor(-0.0028, device='cuda:0')","tensor(-0.0135, device='cuda:0')"


In [ ]:
noise_pattern= '遵照辦理'
noise_pattern1='章節'
noise_pattern2='��'
noise_pattern3='�'
noise_pattern4='修訂處'
noise_pattern5='頁次'

df['text'] = df['text'].replace(noise_pattern,'',regex=True)
df['text'] = df['text'].replace(noise_pattern1,'',regex=True)
df['text'] = df['text'].replace(noise_pattern2,'',regex=True)
df['text'] = df['text'].replace(noise_pattern3,'',regex=True)
df['text'] = df['text'].replace(noise_pattern4,'',regex=True)
df['text'] = df['text'].replace(noise_pattern5,'',regex=True)

In [ ]:
#delete empty rows
df = df[df['text'].str.len()>6]   
#the line below no longer makes sense lol. 
# df = df[['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'accumulated_score']]
df = df[['text', 'original_sequence', 'accumulated_score']]
df = df.reset_index(drop=True)
df

,text,original_sequence,similarity_score1,similarity_score2,similarity_score3,accumulated_score
0,1 \n行政院環境保護署環境影響評估審查委員會 \n第 327 次會議紀錄 \n壹、 時間：...,1,"tensor(0.2263, device='cuda:0')","tensor(0.1678, device='cuda:0')","tensor(0.2463, device='cuda:0')","tensor(1.5695, device='cuda:0')"
1,\n10. 研析因應生態衝擊觀測及共同降載機制,117,"tensor(0.2843, device='cuda:0')","tensor(0.3246, device='cuda:0')","tensor(0.3263, device='cuda:0')","tensor(1.5054, device='cuda:0')"
2,」劉委員小如意見略以：「第 1，簡報 P.8\n鯨豚影響減輕對策中，請確認不得啟動新設風機打...,61,"tensor(0.2211, device='cuda:0')","tensor(0.4175, device='cuda:0')","tensor(0.3045, device='cuda:0')","tensor(1.5050, device='cuda:0')"
3,\n4. 最大噪音量容忍值，參考國際海洋噪音管理與對鯨豚類\n影響減輕規範及國內現有研究調...,206,"tensor(0.5108, device='cuda:0')","tensor(0.2924, device='cuda:0')","tensor(0.2042, device='cuda:0')","tensor(1.4887, device='cuda:0')"
4,（東南） \n（四） P.7-349 表 7.7.5-3 誤植為表 7.7.5-4，請修正,123,"tensor(0.2083, device='cuda:0')","tensor(0.1510, device='cuda:0')","tensor(0.2664, device='cuda:0')","tensor(1.4768, device='cuda:0')"
...,...,...,...,...,...,...
235,\n二、 環境影響摘要 \n（一） 海岸地形變遷模擬結果顯示，4 風場離岸風機設置後對\n...,174,"tensor(0.0718, device='cuda:0')","tensor(0.0012, device='cuda:0')","tensor(0.0227, device='cuda:0')","tensor(0.1328, device='cuda:0')"
236,\n八、 空氣污染 \n施工階段引擎應優先採用低硫氧化物及粒狀污染物等空氣污\n染排放之高...,236,"tensor(0.1150, device='cuda:0')","tensor(-0.0607, device='cuda:0')","tensor(-0.0214, device='cuda:0')","tensor(0.0412, device='cuda:0')"
237,\n（8） 開發影響範圍侷限於場址附近，對其他國家之環境\n無造成顯著不利影響,92,"tensor(-0.0054, device='cuda:0')","tensor(-0.0386, device='cuda:0')","tensor(0.1006, device='cuda:0')","tensor(0.0155, device='cuda:0')"
238,\n9. 開發計畫屬潔淨再生能源風力發電，營運階段於機組\n運轉期間僅以天然風力提供機組運...,48,"tensor(0.0048, device='cuda:0')","tensor(-0.0952, device='cuda:0')","tensor(0.0735, device='cuda:0')","tensor(-0.0135, device='cuda:0')"


In [ ]:
#I used sentiment search to judge whether GPTs output is related to our keywords but I ran into a problem: there is no way to check whether the information is complete. For example, if we set the default threshold
#to 0.7000, and then ask chatGPT to make a summary about certain topics within that text, it will only make a summary about those topics and leave the rest#####
#also, if start with a really high threshold (which may only lead to 2 or 3 sentences) chat gpt will write a perfect summary that SBERT will judge with a high tensor score. This means that the threshold will not go down
#and much information that may also be related to the subject is being left out.

#I therefore thought of an alternative way to create a dynamic threshold. In the code below we find the last row, so the lowest possible tensor score, that contains our 關鍵詞 and delete all the rows after that.
#this way we can be sure that all the important sentences related to our topic will be taken into consideration by GPT for writing its summary or answering our questions. 

# Find the index of the last row containing the word "鯨豚"
index = df[df['text'].str.contains('鯨豚')][::-1].index[0]

# Drop all of the rows after the last row containing the word "鯨豚"
new_df = df.loc[:index]

In [ ]:
new_df

,text,original_sequence,similarity_score1,similarity_score2,similarity_score3,accumulated_score
0,1 \n行政院環境保護署環境影響評估審查委員會 \n第 327 次會議紀錄 \n壹、 時間：...,1,"tensor(0.2263, device='cuda:0')","tensor(0.1678, device='cuda:0')","tensor(0.2463, device='cuda:0')","tensor(1.5695, device='cuda:0')"
1,\n10. 研析因應生態衝擊觀測及共同降載機制,117,"tensor(0.2843, device='cuda:0')","tensor(0.3246, device='cuda:0')","tensor(0.3263, device='cuda:0')","tensor(1.5054, device='cuda:0')"
2,」劉委員小如意見略以：「第 1，簡報 P.8\n鯨豚影響減輕對策中，請確認不得啟動新設風機打...,61,"tensor(0.2211, device='cuda:0')","tensor(0.4175, device='cuda:0')","tensor(0.3045, device='cuda:0')","tensor(1.5050, device='cuda:0')"
3,\n4. 最大噪音量容忍值，參考國際海洋噪音管理與對鯨豚類\n影響減輕規範及國內現有研究調...,206,"tensor(0.5108, device='cuda:0')","tensor(0.2924, device='cuda:0')","tensor(0.2042, device='cuda:0')","tensor(1.4887, device='cuda:0')"
4,（東南） \n（四） P.7-349 表 7.7.5-3 誤植為表 7.7.5-4，請修正,123,"tensor(0.2083, device='cuda:0')","tensor(0.1510, device='cuda:0')","tensor(0.2664, device='cuda:0')","tensor(1.4768, device='cuda:0')"
...,...,...,...,...,...,...
94,主席說明略以：「目前規\n劃之鳥類廊道均為預設，已要求所有離岸風電案必須依\n據今年春季完成...,71,"tensor(0.1424, device='cuda:0')","tensor(0.1901, device='cuda:0')","tensor(0.2107, device='cuda:0')","tensor(0.9667, device='cuda:0')"
95,\n十、 施工前設立環境保護監督小組 \n監督環境影響說明書及審查結論中有關生態保育及環境...,239,"tensor(0.0920, device='cuda:0')","tensor(0.1467, device='cuda:0')","tensor(0.1849, device='cuda:0')","tensor(0.9661, device='cuda:0')"
96,\n（11） 開發單位承諾於施工前設立本案環境保護監督小\n組，監督環境影響說明書及審查結...,21,"tensor(0.0861, device='cuda:0')","tensor(0.1070, device='cuda:0')","tensor(0.2204, device='cuda:0')","tensor(0.9648, device='cuda:0')"
97,\n2. P.6 承諾「評估風場開發所導致的屏障效應」等文字請\n納入本文；另覓食地喪失指...,150,"tensor(0.1184, device='cuda:0')","tensor(0.2331, device='cuda:0')","tensor(0.2234, device='cuda:0')","tensor(0.9645, device='cuda:0')"


## determine threshold

In [ ]:
#we will have to consider how we want to determine the threshold, the simplest way would be to look at the dataframe and check from where it doesn't have anything to do 
#with 噪音 anymore. if there is a more mathematical way to solve this problem I am also open to that. 

# threshold = 0.7710
# mask = df['accumulated_score'].apply(lambda x: x.item() > threshold)

# sbert_out = df[mask]
# sbert_out

## sort by original sequence (optional)

In [ ]:
# sbert_out = sbert_out.sort_values(by='original_sequence').reset_index(drop=True)
# sbert_out=sbert_out['text']

In [ ]:
# sbert_out=sbert_out.to_frame()
# sbert_out

## Summary with GPT 

### prepare data

In [ ]:
import os
import openai
import json

openai.api_key = '' #delete when uploading 

In [ ]:
#disable this cell for sort by orginal sequence 
#fix this this is super confusing lol!!!!
sbert_out=new_df
sbert_out=sbert_out['text']  
sbert_out=sbert_out.to_frame() 
df=sbert_out

In [ ]:
df

,text
0,1 \n行政院環境保護署環境影響評估審查委員會 \n第 327 次會議紀錄 \n壹、 時間：...
1,\n10. 研析因應生態衝擊觀測及共同降載機制
2,」劉委員小如意見略以：「第 1，簡報 P.8\n鯨豚影響減輕對策中，請確認不得啟動新設風機打...
3,\n4. 最大噪音量容忍值，參考國際海洋噪音管理與對鯨豚類\n影響減輕規範及國內現有研究調...
4,（東南） \n（四） P.7-349 表 7.7.5-3 誤植為表 7.7.5-4，請修正
...,...
94,主席說明略以：「目前規\n劃之鳥類廊道均為預設，已要求所有離岸風電案必須依\n據今年春季完成...
95,\n十、 施工前設立環境保護監督小組 \n監督環境影響說明書及審查結論中有關生態保育及環境...
96,\n（11） 開發單位承諾於施工前設立本案環境保護監督小\n組，監督環境影響說明書及審查結...
97,\n2. P.6 承諾「評估風場開發所導致的屏障效應」等文字請\n納入本文；另覓食地喪失指...


In [ ]:
# Find the number of characters in each row of the text column
df['text_length'] = df['text'].str.len()

# Get the cumulative sum of the text lengths
df['cumulative_length'] = df['text_length'].cumsum()

# Find the index of the first row in which the cumulative length is greater than 3600
index = df.index[df['cumulative_length'] > 1500][0]

# Create a new dataframe with only the last few rows
new_df = df.iloc[index:]

# Drop the last few rows from the original dataframe
df = df.iloc[:index]

<ipython-input-88-764434b250fa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_length'] = df['text'].str.len()
<ipython-input-88-764434b250fa>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cumulative_length'] = df['text_length'].cumsum()


In [ ]:
df

,text,text_length,cumulative_length
0,\n四、 決議 \n（一） 「大彰化東北離岸風力發電計畫環境影響說明書」「大\n彰化東南離...,732,732
1,\n二、 李委員堅明 \n（一） 前次意見 1，請將可行性評估內容納入本文,36,768
2,\n（二） 王委員价巨、李委員堅明、劉委員益昌、內政部營建署、\n行政院農業委員會漁業署...,201,969
3,\n○4 鯨豚：4 案風場均非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音...,61,1030
4,」 \n（三） 經濟部能源局代表發言如附件 6,22,1052
5,\n十、 本署環境督察總隊 \n（一） 前次會議結論三（二）1.「距離打樁位置外 750 ...,159,1211
6,\n3. 開發單位就專案小組所提下列主要意見，已承諾納入辦\n理，請據以補充、修正環境影響...,287,1498


In [ ]:
sbert_out=df['text']
# sbert_out_1=new_df['text']
# sbert_out_1 = sbert_out_1.reset_index(drop=True)
# sbert_out.loc[0] = ['第327次會議是什麼時候？ 第327次會議中，對於鯨豚保育的重點是什麼？ 第327次會議中，有哪些專家為鯨豚保育發言？']
# sbert_out_1.loc[0] = ['第327次會議是什麼時候？ 第327次會議中，對於鯨豚保育的重點是什麼？ 第327次會議中，有哪些專家為鯨豚保育發言？']
sbert_out=sbert_out.to_frame()


In [ ]:
sbert_out

,text
0,\n四、 決議 \n（一） 「大彰化東北離岸風力發電計畫環境影響說明書」「大\n彰化東南離...
1,\n二、 李委員堅明 \n（一） 前次意見 1，請將可行性評估內容納入本文
2,\n（二） 王委員价巨、李委員堅明、劉委員益昌、內政部營建署、\n行政院農業委員會漁業署...
3,\n○4 鯨豚：4 案風場均非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音...
4,」 \n（三） 經濟部能源局代表發言如附件 6
5,\n十、 本署環境督察總隊 \n（一） 前次會議結論三（二）1.「距離打樁位置外 750 ...
6,\n3. 開發單位就專案小組所提下列主要意見，已承諾納入辦\n理，請據以補充、修正環境影響...


In [ ]:
# sbert_text = sbert_out['text'].to_string(index=False) #this leads to incomplete text. use join
sbert_text = " ".join(sbert_out['text'].tolist())
len(sbert_text)

In [ ]:
sbert_text = re.sub(r'[\n\\]+', '', sbert_text)
sbert_text= re.sub(r'n|n－', '', sbert_text)
sbert_text

' 四、 決議 （一） 「大彰化東北離岸風力發電計畫環境影響說明書」「大彰化東南離岸風力發電計畫環境影響說明書」「大彰化西北離岸風力發電計畫環境影響說明書」「大彰化西南離岸風力發電計畫環境影響說明書」等 4 案審查結論如下： 1. 經綜合考量環境影響評估審查委員會委員、專家學者、各方意見及開發單位之答覆，就「大彰化東北離岸風力發電計畫環境影響說明書」「大彰化東南離岸風力發電計畫環境影響說明書」「大彰化西北離岸風力發電計畫環境影響說明書」「大彰化西南離岸風力發電計畫環境影響說明書」等 4 案（下簡稱 4 案）生活環境、自然環境、社會環境及經濟、文化、生態等可能累積加乘影響之程度及範圍，經專業判斷，認定已無環境影響評估法第 8 條及施行細則第 19 條第 1 項第 1 款及第 2 款所列各目情形之虞，環境影響說明書已足以提供審查判斷所需資訊，無須進行第二階段環境影響評估，評述理由如下： （1） 就 4 案開發行為包括環境影響評估法施行細則第19 條第 1 項第 1 款附表二之「345 千伏或 161 千伏輸電線路架空或地下化線路鋪設長度 50 公里以上者」，考量開發單位採行高電壓輸出海纜，減少海27 纜鋪設數量或範圍，施工方式除潮間帶採水平導向式鑽掘(HDD)，其餘海纜範圍採犁埋機或噴埋機，配合海纜鋪設完成後海床沉積物隨即自然覆蓋，開發單位承諾依「離岸風電區塊開發政策評估說明 書」本署徵詢意見採行因應對策，海纜上岸路線規劃於台灣電力股份有限公司依經濟部 106 年 8 月 2日經能字第 10602611030 號函公告「彰化離岸風電海纜上岸共同廊道範圍」之北側廊道，以減輕整體環境影響  二、 李委員堅明 （一） 前次意見 1，請將可行性評估內容納入本文  （二）  王委員价巨、李委員堅明、劉委員益昌、內政部營建署、行政院農業委員會漁業署、行政院海岸巡防署、文化部文化資產局、本署綜合計畫處、環境督察總隊等意見經開發單位於會中說明，業經本會確認，請開發單位將補充說明資料及以下內容納入定稿： 31 1. 鳥類通行廊道之規劃，應俟完成 106 年秋季至 107 年春季鳥類環境影響調查報告，並依環境影響評估法第 18條規定完成審查後方予定案  ○4 鯨豚：4 案風場均非位於中華白海豚野生動物重要棲息環境預告範圍，並依水下噪音模擬評估結果，已擬定海豚保護措施 」 （三） 經濟部能源

In [ ]:
len(sbert_text)

1438

### prompt GPT

In [ ]:
summary = ''

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= '第327次會議是什麼時候？'+sbert_text,
  temperature=0.9,
  max_tokens=300,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=[" Human:", " AI:"]
  )
response
json_object = json.loads(str(response))
text = json_object['choices'][0]['text']
summary = summary + text

summary

'，噪音閾值不得超出德國 StUK4(2013)的環評標準[1]。\n\n第327次會議的時間尚未公佈。'

In [ ]:
summary1 = ''

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= ' 第327次會議中，對於鯨豚保育的重點是什麼？'+sbert_text,
  temperature=0.9,
  max_tokens=300,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=[" Human:", " AI:"]
  )
response
json_object = json.loads(str(response))
text = json_object['choices'][0]['text']
summary1 = summary1 + text

summary1

'，以評估機組施工期施工對海生物及海域水理之影響，對於超過持續營運級聲學分析測量閾值之部分，應採取減緩、改善或停止施工措施\n\n第327次會議中，對於鯨豚保育的重點是以採取減緩、改善或停止施工措施來保護海洋中的鯨豚物種。尤其是要監測打樁水下噪音值，當超過持續營運級聲學分'

In [ ]:
 #remove key when uploading to github
summary2 = ''

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= '第327次會議中，有哪些專家為鯨豚保育發言？'+sbert_text,
  temperature=0.9,
  max_tokens=300,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=[" Human:", " AI:"]
  )
response
json_object = json.loads(str(response))
text = json_object['choices'][0]['text']
summary2 = summary2 + text

summary2

'\n\n1. 行政院環境保護署環境影響評估審查委員會第327次會議中發言關於鯨豚保育的專家包括：張委員希平、劉委員小如意和劉委員希平。'

In [ ]:
print('第327次會議是什麼時候？'+ summary)
print('第327次會議中，對於鯨豚保育的重點是什麼？'+ summary1)
print('第327次會議中，有哪些專家為鯨豚保育發言？'+ summary2)

第327次會議是什麼時候？ （五） P.7-346 表 7.7.5-3 誤植為表 7.7.5-4，請修正                        

第 327 次會議是106 年 11 月 27 日舉行的。
第327次會議中，對於鯨豚保育的重點是什麼？ 
答：第327次會議中，針對鯨豚保育的重點是對減輕對鯨豚影響的條件和措施進行研究和檢討，包括加強監測和配備觀察船，提高相關科技技術水準，安裝水下聲學監測設施，配置鯨豚生態觀察員，建立共同降載機制等。
第327次會議中，有哪些專家為鯨豚保育發言？

1. 行政院環境保護署環境影響評估審查委員會第327次會議中發言關於鯨豚保育的專家包括：張委員希平、劉委員小如意和劉委員希平。


## adjust answers based on more information 

### prepare additional data

In [ ]:
new_df

,text,text_length,cumulative_length
25,\n四、 決議 \n（一） 「大彰化東北離岸風力發電計畫環境影響說明書」「大\n彰化東南離...,732,3710
26,\n二、 李委員堅明 \n（一） 前次意見 1，請將可行性評估內容納入本文,36,3746
27,\n（二） 王委員价巨、李委員堅明、劉委員益昌、內政部營建署、\n行政院農業委員會漁業署...,201,3947
28,\n○4 鯨豚：4 案風場均非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音...,61,4008
29,」 \n（三） 經濟部能源局代表發言如附件 6,22,4030
...,...,...,...
94,主席說明略以：「目前規\n劃之鳥類廊道均為預設，已要求所有離岸風電案必須依\n據今年春季完成...,94,11183
95,\n十、 施工前設立環境保護監督小組 \n監督環境影響說明書及審查結論中有關生態保育及環境...,198,11381
96,\n（11） 開發單位承諾於施工前設立本案環境保護監督小\n組，監督環境影響說明書及審查結...,210,11591
97,\n2. P.6 承諾「評估風場開發所導致的屏障效應」等文字請\n納入本文；另覓食地喪失指...,199,11790


In [ ]:
# 補充answer with other rows?
# reset index
new_df=new_df['text']
new_df = new_df.reset_index(drop=True)


In [ ]:
new_df=new_df.to_frame()

In [ ]:
df=new_df

In [ ]:
# Find the number of characters in each row of the text column
df['text_length'] = df['text'].str.len()

# Get the cumulative sum of the text lengths
df['cumulative_length'] = df['text_length'].cumsum()

# Find the index of the first row in which the cumulative length is greater than 3600
index = df.index[df['cumulative_length'] > 3600][0]

# Create a new dataframe with only the last few rows
new_df = df.iloc[index:]

# Drop the last few rows from the original dataframe
df = df.iloc[:index]

In [ ]:
df

,text,text_length,cumulative_length
0,\n四、 決議 \n（一） 「大彰化東北離岸風力發電計畫環境影響說明書」「大\n彰化東南離...,732,732
1,\n二、 李委員堅明 \n（一） 前次意見 1，請將可行性評估內容納入本文,36,768
2,\n（二） 王委員价巨、李委員堅明、劉委員益昌、內政部營建署、\n行政院農業委員會漁業署...,201,969
3,\n○4 鯨豚：4 案風場均非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音...,61,1030
4,」 \n（三） 經濟部能源局代表發言如附件 6,22,1052
5,\n十、 本署環境督察總隊 \n（一） 前次會議結論三（二）1.「距離打樁位置外 750 ...,159,1211
6,\n3. 開發單位就專案小組所提下列主要意見，已承諾納入辦\n理，請據以補充、修正環境影響...,287,1498
7,\n（九） P.2 審查結論（一）1 之答覆說明，請確實承諾：「在距\n離打樁位置外 75...,100,1598
8,\n陸、 討論事項 \n第一案 \n雲林離岸風力發電廠興建計畫環境影響說明書 \n一、 本...,267,1865
9,\n3. 簡報 P.10 打樁期間承諾「不使用聲學裝置(ADD)」，請\n納入報告書第 8...,48,1913


In [ ]:
sbert_out_1=df['text']
sbert_out_1=sbert_out_1.to_frame()
sbert_out_1

,text
0,\n四、 決議 \n（一） 「大彰化東北離岸風力發電計畫環境影響說明書」「大\n彰化東南離...
1,\n二、 李委員堅明 \n（一） 前次意見 1，請將可行性評估內容納入本文
2,\n（二） 王委員价巨、李委員堅明、劉委員益昌、內政部營建署、\n行政院農業委員會漁業署...
3,\n○4 鯨豚：4 案風場均非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音...
4,」 \n（三） 經濟部能源局代表發言如附件 6
5,\n十、 本署環境督察總隊 \n（一） 前次會議結論三（二）1.「距離打樁位置外 750 ...
6,\n3. 開發單位就專案小組所提下列主要意見，已承諾納入辦\n理，請據以補充、修正環境影響...
7,\n（九） P.2 審查結論（一）1 之答覆說明，請確實承諾：「在距\n離打樁位置外 75...
8,\n陸、 討論事項 \n第一案 \n雲林離岸風力發電廠興建計畫環境影響說明書 \n一、 本...
9,\n3. 簡報 P.10 打樁期間承諾「不使用聲學裝置(ADD)」，請\n納入報告書第 8...


In [ ]:
sbert_text_1 = sbert_out_1['text'].to_string(index=False)

In [ ]:
sbert_text_1 = re.sub(r'[\n\\]+', '', sbert_text_1)
sbert_text_1 = re.sub(r'n|n－', '', sbert_text_1)
sbert_text_1

' 四、 決議 （一） 「大彰化東北離岸風力發電計畫環境影響說明書」「大彰化東南離岸...             二、 李委員堅明 （一） 前次意見 1，請將可行性評估內容納入本文 （二）  王委員价巨、李委員堅明、劉委員益昌、內政部營建署、行政院農業委員會漁業署、... ○4 鯨豚：4 案風場均非位於中華白海豚野生動物重要棲息環境預告範圍，並依水下噪音模...                           」 （三） 經濟部能源局代表發言如附件 6 十、 本署環境督察總隊 （一） 前次會議結論三（二）1.「距離打樁位置外 750 公... 3. 開發單位就專案小組所提下列主要意見，已承諾納入辦理，請據以補充、修正環境影響說... （九） P.2 審查結論（一）1 之答覆說明，請確實承諾：「在距離打樁位置外 750... 陸、 討論事項 第一案 雲林離岸風力發電廠興建計畫環境影響說明書 一、 本署... 3. 簡報 P.10 打樁期間承諾「不使用聲學裝置(ADD)」，請納入報告書第 8 章本文  69 3. 打樁前應先確認至少 30 分鐘無鯨豚活動後方得作業；施工過程若...                           」 （七） 經濟部能源局代表發言如附件 6   54 「大彰化東北離岸風力發電計畫環境影響說明書」「大彰化東南離岸風... 18 4. 建議目的事業主管機關經濟部能源局辦理以下事項： （1） 依 106 ...        （3） 倘本署訂定水下噪音標準檢測方法或更嚴格之管制標準時，則應依該規定辦理 （四） 經比對本報告書與 106 年 11 月 27 日本案專案小組第 3次初審會議...第 2，簡報 P.22，4 案均承諾會分別成立環境保護監督小組，且就『專家學者』『民間... （二） 為降低機組開發工程之可能影響，如必須使用敲擊式基樁，考量選擇較細或採多支基座... （五） 在水下噪音及對鯨豚影響部分，依據水下噪音模擬評估結果，在採用管架式基樁基礎及...                 7. 內政部依海岸管理法訂定或公布之「一、二級海岸保護區」 7. 協助與科技部、文化部及教育部等有關部會，藉由我國離岸風力發電計畫推動契機，建構... 6. 文化部依水下文化資產保存法劃設之「水下文化資產保護區」與依法列冊及管理疑似水下...」

### prompt GPT (part 2)

In [ ]:
summary3 = ''

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= '我剛剛問你：第327次會議中，對於鯨豚保育的重點是什麼？你的答案就是'+ summary1+ '請以之下的資料補充你之前的答案答案'+sbert_text_1,
  temperature=0.9,
  max_tokens=300,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=[" Human:", " AI:"]
  )
response
json_object = json.loads(str(response))
text = json_object['choices'][0]['text']
summary3 = summary3 + text

summary3

' \n答:第327次會議中，針對鯨豚保育的重點是：\n1. 確認不得啟動新設風機打樁作業；\n2. 最大噪音容忍值參考國際海洋噪音管理及對鯨豚類影響減輕規範；\n3. 施工船上配置至少3位以上鯨豚觀測員，1位需為民間生態團體成員；\n4. 納入生態衝擊觀測及共同降載機制；\n5. 設置水下聲學監測；\n6'